# MNIST CLASSIFICATION

### Imports

In [1]:
import tensorflow as tf

import deel.lipdp.layers as DP_layers
import deel.lipdp.losses as DP_losses
from deel.lipdp.pipeline import bound_clip_value
from deel.lipdp.pipeline import load_and_prepare_data
from deel.lipdp.sensitivity import get_max_epochs
from deel.lipdp.model import DP_Accountant
from deel.lipdp.model import DP_Sequential
from deel.lipdp.model import DPParameters
from deel.lipdp.model import AdaptiveLossGradientClipping

2023-06-30 14:20:36.337516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 14:20:37.893701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the data :

It is important to import the data with the right DP parameters to account properly for the privacy guarantees of the trained model.

In [7]:
ds_train, ds_test, dataset_metadata = load_and_prepare_data(
    "mnist",
    batch_size=500,
    drop_remainder=True,  # accounting assumes fixed batch size
    bound_fct=bound_clip_value(
        10.0
    ),  # clipping preprocessing allows to control input bound
)

: 

### Declaring the DP parameters :

We also need to declare explicitly the parameters of the DP training process.

In [3]:
dp_parameters = DPParameters(
    noisify_strategy="global",
    noise_multiplier=2.0,
    delta=1e-5,
)

### Defining the model :

We use a simple convolutive network to classify on the MNIST dataset. We add a loss gradient clipping layer at the end of our network for more tightness on our gradient's upper bound. Therefore allowing for better results with one less hyperparameter to tune for dynamically chosen clipping constant. 

In [4]:
layers = [
    DP_layers.DP_BoundedInput(
        input_shape=dataset_metadata.input_shape,
        upper_bound=dataset_metadata.max_norm,
    ),
    DP_layers.DP_SpectralConv2D(
        filters=16, kernel_size=3, use_bias=False, kernel_initializer="orthogonal"
    ),
    DP_layers.DP_Flatten(),
    DP_layers.DP_SpectralDense(
        units=10, use_bias=False, kernel_initializer="orthogonal"
    ),
    DP_layers.DP_ClipGradient(
        epsilon=1, mode="dynamic_svt", patience=5
    )
]

model = DP_Sequential(
    layers=layers, dp_parameters=dp_parameters, dataset_metadata=dataset_metadata
)

loss = DP_losses.DP_TauCategoricalCrossentropy(14.5)

# Compatible with any kind of non-private optimizer : 
opt = tf.keras.optimizers.SGD(learning_rate=1e-2)

model.compile(
    loss=loss,
    optimizer=opt,
    metrics=["accuracy"],
    run_eagerly=False,
)

/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/deel/lip/model.py:74: UserWarning: Sequential model contains a layer which is not a 1-Lipschitz layer: dp__bounded_input
  warn(_msg_not_lip.format(layer.name))
/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/deel/lip/model.py:74: UserWarning: Sequential model contains a layer which is not a 1-Lipschitz layer: dp__clip_gradient
  warn(_msg_not_lip.format(layer.name))


### Define the desired DP guarantees :

We compute the budget of epochs needed to yields the DP guarantees that you desire :

In [5]:
num_epochs = get_max_epochs(3.0, model)

/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/scipy/optimize/_optimize.py:2769: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


epoch bounds = (0, 512.0) and epsilon = 85.26272177783693 at epoch 512.0
epoch bounds = (0, 256.0) and epsilon = 48.25957389491231 at epoch 256.0
epoch bounds = (0, 128.0) and epsilon = 26.21744943567399 at epoch 128.0
epoch bounds = (0, 64.0) and epsilon = 13.425955622771932 at epoch 64.0
epoch bounds = (32.0, 64.0) and epsilon = 7.139827972307393 at epoch 32.0
epoch bounds = (32.0, 48.0) and epsilon = 10.318302386220275 at epoch 48.0
epoch bounds = (32.0, 40.0) and epsilon = 9.224465968334444 at epoch 40.0
epoch bounds = (32.0, 36.0) and epsilon = 8.185134158355854 at epoch 36.0
epoch bounds = (34.0, 36.0) and epsilon = 7.178060508347013 at epoch 34.0
epoch bounds = (34.0, 35.0) and epsilon = 8.167545918253115 at epoch 35.0


### Train the model : 

The training process is called through the model.fit attribute. We use the following callbacks : 

- **DP_Accountant** (log_fn) : accounts for the privacy guarantees after each epoch of training (*log_fn* makes it compatible with W&B logging).
- **DP_AdaptiveGradientClipping** (ds_train, patience) : automatically updates the losses's gradient clipping constant every *patience* steps. 


In [6]:
callbacks = [
    DP_Accountant(log_fn="logging"),
    AdaptiveLossGradientClipping(
        ds_train=ds_train
    ),  # DO NOT USE THIS CALLBACK WHEN mode != "dynamic_svt"
]

hist = model.fit(
    ds_train,
    epochs=num_epochs,
    validation_data=ds_test,
    callbacks=callbacks,
)

On train begin : 
Initial value is now equal to lipschitz constant of loss:  tf.Tensor(1.4142135, shape=(), dtype=float32)
Epoch 1/35


2023-06-30 14:20:44.284755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-06-30 14:20:44.285101: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-06-30 14:20:46.041955: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-30 14:20:46.693473: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.1.0 but sourc

UnimplementedError: Graph execution error:

Detected at node 'dp__sequential/dp__spectral_conv2d/convolution' defined at (most recent call last):
    File "/opt/miniconda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/miniconda/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/to

### 